In [4]:
import re
from typing import List, Tuple
from urllib.parse import urljoin

import requests
from bs4 import BeautifulSoup
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.tools import Tool
from dotenv import load_dotenv
import os

load_dotenv()


True

In [5]:
class FalabellaScraper:
    def __init__(self, openai_api_key: str):
        self.base_url = "https://www.falabella.com/falabella-cl/page/"
        self.visited_urls = set()
        self.results = []

        # Initialize LangChain agent
        llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo", openai_api_key=openai_api_key)
        tools = [
            Tool(
                name="Group Text",
                func=self._group_text,
                description="Groups text under appropriate headers",
            )
        ]
        self.agent = initialize_agent(tools, llm, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

    def scrape(self, start_url: str) -> List[Tuple[str, str]]:
        self._scrape_recursive(start_url)
        return self.results
    
    def scrape_page(self, url: str) -> Tuple[str, str]:
        response = requests.get(url, timeout=30)  # Add a 30-second timeout
        soup = BeautifulSoup(response.text, 'html.parser')
        text_content = soup.get_text(separator='\n', strip=True)
        grouped_text = self.agent.run(f"Group the following text under appropriate headers:\n\n{text_content}")
        return url, grouped_text

    def _scrape_recursive(self, url: str):
        if url in self.visited_urls:
            return

        self.visited_urls.add(url)
        response = requests.get(url, timeout=30)  # Add a 30-second timeout
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract text content
        text_content = soup.get_text(separator='\n', strip=True)

        # Use LangChain agent to group text
        grouped_text = self.agent.run(f"Group the following text under appropriate headers:\n\n{text_content}")
        self.results.append((url, grouped_text))

        # Find and follow links
        for link in soup.find_all('a', href=True):
            href = link['href']
            full_url = urljoin(url, href)
            if full_url.startswith(self.base_url) and full_url not in self.visited_urls:
                self._scrape_recursive(full_url)

    def _group_text(self, text: str) -> str:
        # This is a placeholder for the actual grouping logic
        # The LangChain agent will use this function to group the text
        return text

In [6]:
start_url = "https://www.falabella.com/falabella-cl/page/contactanos"

scraper = FalabellaScraper(openai_api_key=os.environ["OPENAI_API_KEY"])




/tmp/ipykernel_12600/3858496109.py:8: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo", openai_api_key=openai_api_key)
/tmp/ipykernel_12600/3858496109.py:16: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  self.agent = initialize_agent(tools, llm, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)


In [8]:
response = requests.get("https://www.falabella.com/falabella-cl/page/cambios-reemplazos", timeout=30)  # Add a 30-second timeout
soup = BeautifulSoup(response.text, 'html.parser')
text_content = soup.get_text(separator='\n', strip=True)
text_content
# grouped_text = self.agent.run(f"Group the following text under appropriate headers:\n\n{text_content}")

'Cambiar o reemplazar un producto | falabella.com\nMenú\nSearch Bar\nHola\n,\nInicia sesión\nMis\ncompras\n0\nlocation-icon\nIngresa tu ubicación\nVende en falabella.com\nTarjetas y cuentas\nVenta telefónica\nNovios\nAyuda\nHome\nCambiar o reemplazar un producto\nCargando...\nCambiar o reemplazar un producto\nEn falabella.com te acompañamos este fin de año y ampliamos el plazo para cambios de productos comprados en:\nCyber Monday (realizado entre el 30 de septiembre y el 02 de octubre)\ntienen hasta el 31 de enero del 2025\n.\nPeriodo navidad (compras entre el 01 de noviembre y el 24 de diciembre)\ntienen hasta el 31 de enero del 2025\n.\nCambiar de opinión está bien. Gracias a nuestra Satisfacción Garantizada,\ncon la mayoría de los productos tienes 30 días desde que los recibes\npara pedir un cambio o reemplazo.\nDe todas formas, considera que algunos productos cuentan con\notros plazos, restricciones y excepciones\n.\nProducto pequeño o mediano\nPor ejemplo: perfumes, zapatos, micro

In [7]:
results = scraper.scrape(start_url)
results

/tmp/ipykernel_12600/3858496109.py:34: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  grouped_text = self.agent.run(f"Group the following text under appropriate headers:\n\n{text_content}")




> Entering new AgentExecutor chain...
Question: Group the following text under appropriate headers.

Thought: The text contains various sections related to customer service, contact information, different services offered, and company information. We can group them under relevant headers for better organization.

Action:
```
{
  "action": "Group Text",
  "action_input": "Centro de ayuda | falabella.com Menú Search Bar Hola , Inicia sesión Mis compras 0 location-icon Ingresa tu ubicación Vende en falabella.com Tarjetas y cuentas Venta telefónica Novios Ayuda Home Centro de ayuda Cargando... Centro de Ayuda Revisa y gestiona tus pedidos Sigue tus compras. Cancela tus pedidos. Haz tus solicitudes de devolución. Revisa tus boletas y tickets de cambio. Ingresa tus casos. Ir a Mis compras ¿Aún necesitas ayuda? Contáctanos Llámanos 600 329 2002. Hablemos de lunes a sábados de 09:00 a 19:00 h. Llamar a Servicio al cliente Hablemos por WhatsApp Escríbenos de lunes a sábados de 09:00 a 19:00 h

ToolException: Too many arguments to single-input tool Group Text.
                Consider using StructuredTool instead. Args: [['Menú', 'Search Bar', 'Hola', 'Inicia sesión', 'Mis compras', 'location-icon', 'Ingresa tu ubicación'], ['+180 millones de visitas mensuales regionales', 'Crecimiento de las ventas en x2,8 veces desde 2019', '37 millones de clientes a nivel regional'], ['Disfruta de todos los beneficios de vender en falabella.com:', 'Mayor visibilidad para tu marca', 'Herramientas de análisis de datos', 'Servicios logísticos potenciados', 'Campañas masivas en medios', 'Alternativas de financiamiento para tu negocio', 'Servicios adicionales de marketing'], ['Caso de éxito: Amamantas', 'Caso de éxito: Tomasstore'], [], ['Autogestionable y llena de herramientas para tomar las mejores decisiones', 'Creación de productos', 'Integraciones', 'Gestión de Órdenes', 'Operación logística', 'Gestión de pagos', 'Análisis de métricas y herramientas de pricing', 'Campañas', 'Guías de ayuda

In [ ]:
start_url = "https://www.falabella.com/falabella-cl/page/contactanos"

scraper = FalabellaScraper(openai_api_key=os.environ["OPENAI_API_KEY"])
results = scraper.scrape(start_url)

for url, grouped_text in results:
    print(f"URL: {url}")
    print("Grouped Text:")
    print(grouped_text)
    print("-" * 50)